#### Imports

In [1]:
import requests
import datetime
import base64
from pathlib import Path
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import padding
from config import (
    KALSHI_API_KEY,
    KALSHI_PRIVATE_KEY_PATH,
    KALSHI_BASE_URL,
    POLYMARKET_API_KEY,
    POLYMARKET_BASE_URL,
    validate_config
)

#### Connect to Kalshi

In [2]:
validate_config()

# Load private key
with open(Path(KALSHI_PRIVATE_KEY_PATH), 'rb') as f:
    kalshi_private_key = serialization.load_pem_private_key(
        f.read(), password=None, backend=default_backend()
    )

def kalshi_get(path):
    """Make authenticated GET request to Kalshi API."""
    timestamp = str(int(datetime.datetime.now().timestamp() * 1000))
    full_path = f"/trade-api/v2{path}"
    
    message = f"{timestamp}GET{full_path}".encode('utf-8')
    signature = kalshi_private_key.sign(
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.DIGEST_LENGTH),
        hashes.SHA256()
    )
    
    headers = {
        'KALSHI-ACCESS-KEY': KALSHI_API_KEY,
        'KALSHI-ACCESS-SIGNATURE': base64.b64encode(signature).decode('utf-8'),
        'KALSHI-ACCESS-TIMESTAMP': timestamp
    }
    
    return requests.get(KALSHI_BASE_URL + path, headers=headers)

# Test connection
response = kalshi_get("/portfolio/balance")
print(f"{'✓' if response.status_code == 200 else '✗'} Kalshi status: {response.status_code}")

✓ Kalshi status: 200


##### Connect to Polymarket

In [3]:
def polymarket_get(path):
    """Make GET request to Polymarket API."""
    headers = {
        "Authorization": f"Bearer {POLYMARKET_API_KEY}",
        "Content-Type": "application/json"
    }
    return requests.get(POLYMARKET_BASE_URL + path, headers=headers)

# Test connection
response = polymarket_get("/markets")
print(f"{'✓' if response.status_code == 200 else '✗'} Polymarket status: {response.status_code}")

✓ Polymarket status: 200


#### List Holding Details for Kalshi

In [4]:
def get_kalshi_holdings():
    """Get Kalshi account balance and positions."""
    balance = kalshi_get("/portfolio/balance").json()
    print("=== Balance ===")
    print(f"Available: ${balance['balance'] / 100:.2f}")
    print()
    
    positions = kalshi_get("/portfolio/positions?settlement_status=unsettled").json()
    print("=== Positions ===")
    if positions.get('market_positions'):
        print(f"Active positions: {len(positions['market_positions'])}\n")
        for i, pos in enumerate(positions['market_positions'][:5], 1):
            print(f"{i}. {pos['ticker']} - Position: {pos['position']}, Traded: {pos['total_traded']}")
    else:
        print("No active positions")

get_kalshi_holdings()

=== Balance ===
Available: $9.50

=== Positions ===
No active positions
